In [1]:
from typing import List
import os

In [2]:
DUMMY_WORDSEARCH = "MMMSXXMASM\nMSAMXMSMSA\nAMXSXMAAMM\nMSAMASMSMX\nXMASAMXAMM\nXXAMMXXAMA\nSMSMSASXSS\nSAXAMASAAA\nMAMMMXMMMM\nMXMXAXMASX"

## Part 1

In [3]:
def test_word_searcher(func):
    # the word xmas appears 18 times in valid formats
    expected_output = 18

    function_output = func(DUMMY_WORDSEARCH, "XMAS")

    if function_output != expected_output:
        raise ValueError("function does not return correct value")
    else:
        print("passed")


In [4]:
def test_word_counter(func):
    dummy_input = "ababccabc"
    expected_output = 2

    function_output = func(dummy_input, "abc")

    if function_output != expected_output:
        raise ValueError("function does not return correct value")
    else:
        print("passed")

In [5]:
def word_counter(string: str, keyword: str) -> int:
    """
    Counts how many times a keyword occurs in a string in one direction.
    """
    if keyword not in string:
        return 0
    
    return len(string.split(keyword)) - 1 # splitting a string on another string obtains n+1 splits
    

In [6]:
test_word_counter(word_counter)

passed


In [ ]:
def get_strings(string: str, min_length:int) -> List[str]:
    """
    Gets all possible linear strings from a wordsearch grid.
    """
    # get a list of all rows 
    rows = string.split("\n")
    num_rows = len(rows)
    num_cols = len(rows[0])

    if not all(len(row) == num_cols for row in rows):
        raise ValueError("rows are not of equal lengths")
    
    # get a list of all columns
    cols = [''.join(column) for column in zip(*rows)]

    # get a list of all diagnals
    diagonals = []

    # note: diagonal lines are characterised by the difference between column and row number 
    min_diff = -(num_rows - min_length)
    max_diff = num_cols - min_length

    for diff in range(min_diff, max_diff + 1):
        down_diag = []
        up_diag = []

        for col_num in range(max(diff, 0), min(num_rows + diff, num_cols)):
            row_num = col_num - diff
            down_diag.append(rows[row_num][col_num])
            up_diag.append(rows[row_num][num_cols - col_num - 1])

        diagonals.append("".join(down_diag))  
        diagonals.append("".join(up_diag))  
        
    return rows + cols + diagonals

In [ ]:
def word_searcher(wordsearch: str, keyword) -> int:
    """
    Takes a wordsearch grid and returns the number of times a keyword occurs in a valid format.
    """

    strings_to_check = get_strings(wordsearch, len(keyword))

    count = 0
    for string in strings_to_check:
        count += word_counter(string, keyword)
        count += word_counter(string, keyword[::-1])

    return count

In [9]:
test_word_searcher(word_searcher)

passed


In [10]:
with open('input_4.txt') as file:
    wordsearch = file.read()  

In [11]:
word_searcher(wordsearch, "XMAS")

2507

## Part 2

In [12]:
def test_X_word_searcher(func):
    # the word mas occurs in an x 9 times
    expected_output = 9

    function_output = func(DUMMY_WORDSEARCH, "MAS")

    if function_output != expected_output:
        raise ValueError("function does not return correct value")
    else:
        print("passed")

In [13]:
def test_find_word_centers(func):
    dummy_input = "MMMASABCMASMASABCMASABC"
    expected_output = [3, 9, 12, 18]

    function_output = func(dummy_input, "MAS")

    if function_output != expected_output:
        raise ValueError("function does not return correct value")
    else:
        print("passed")

In [14]:
def find_word_centers(string: str, keyword: str) -> List[int]:
    """
    Obtains the center positions (zero indexed) of keywords in string if they exist in that string
    """
    if len(keyword) % 2 == 0:
        raise ValueError("key word must have an odd number of letters")

    if keyword not in string:
        return []
    
    centers = []
    for substr in string.split(keyword)[:-1]:
        if len(centers) > 0:
            letters_so_far = 1 + centers[-1] + (len(keyword) // 2)
        else:
            letters_so_far = 0
        centers.append(letters_so_far + len(substr) +  (len(keyword) // 2)) 
        # note: we add one to account for the middle letter and subtract one to account for zero-indeximg
    return centers


In [15]:
test_find_word_centers(find_word_centers)

passed


In [78]:
def X_word_searcher(wordsearch: str, keyword: str) -> int: 
    """
    Takes a wordsearch grid and returns the number of times a keyword occurs in an X shape.
    """
    # get a list of all rows 
    rows = wordsearch.split("\n")
    num_rows = len(rows)
    num_cols = len(rows[0])

    if not all(len(row) == num_cols for row in rows):
        raise ValueError("rows are not of equal lengths")

    # initialise a list of coordinates for the centers of upwards and downward matches
    all_up_centers = []
    all_down_centers = []

    # note: diagonal lines are characterised by the difference between column and row number
    min_length = len(keyword)
    min_diff = -(num_rows - min_length)
    max_diff = num_cols - min_length

    for diff in range(min_diff, max_diff + 1):
        down_diag = []
        up_diag = []

        for col_num in range(max(diff, 0), min(num_rows + diff, num_cols)):
            row_num = col_num - diff
            down_diag.append(rows[row_num][col_num])
            up_diag.append(rows[row_num][num_cols - col_num - 1])
        
        # make a string for each down diagonal and save the center coordinates of each keyword match (and reversed)
        # this should match up with the center of an up diagonal word to form an X
        down_diag = "".join(down_diag)
        down_centers = find_word_centers(down_diag, keyword) + find_word_centers(down_diag, keyword[::-1])
        down_centers = [(center + max(diff, 0), center - min(diff, 0)) for center in down_centers] # convert linear position to coordinates in the grid
        all_down_centers += down_centers

        up_diag = "".join(up_diag)
        up_centers = find_word_centers(up_diag, keyword) + find_word_centers(up_diag, keyword[::-1])
        up_centers = [(num_cols - center - max(diff, 0) - 1, center - min(diff, 0)) for center in up_centers]
        all_up_centers += up_centers

    all_up_centers = set(all_up_centers)
    all_down_centers = set(all_down_centers)

    # take only centers that are shared by an up and down diagonal i.e. form an X
    matching_centers = all_up_centers & all_down_centers

    return len(matching_centers)

In [79]:
test_X_word_searcher(X_word_searcher)

passed


In [81]:
X_word_searcher(wordsearch, "MAS")

1969